In [1]:
from selenium import webdriver

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

import pandas as pd

https://s.cafef.vn/bao-cao-tai-chinh/FPT/BSheet/2022/0/0/0/bao-cao-tai-chinh-cong-ty.chn

https://s.cafef.vn/bao-cao-tai-chinh/FPT/IncSta/2022/0/0/0/ket-qua-hoat-dong-kinh-doanh-cong-ty.chn

https://s.cafef.vn/bao-cao-tai-chinh/FPT/CashFlow/2022/0/0/0/ket-qua-hoat-dong-kinh-doanh-cong-ty.chn

### Define Functions

In [31]:
def get_elements_value(items, convert_numeric=True):
    list = []
    if convert_numeric==False:
        for x in items:
            list.append(x.text)
    else:
        for x in items:
            num = x.text.strip().replace(',','')
            if num.isnumeric():
                list.append(int(num))
            else:
                list.append(num)
    return list

def year_col_process(driver, col):
    x_path = f"//*[@id='tableContent']/tbody/tr/td[{col}]"
    rows = driver.find_elements(By.XPATH, x_path)
    return rows

def get_report_url(ticker, year, report_type):
    if report_type.upper() == 'BS':
        return f"https://s.cafef.vn/bao-cao-tai-chinh/{ticker}/BSheet/{year}/0/0/0/bao-cao-tai-chinh-cong-ty.chn"
    if report_type.upper() == 'IS':
        return f"https://s.cafef.vn/bao-cao-tai-chinh/{ticker}/IncSta/{year}/0/0/0/ket-qua-hoat-dong-kinh-doanh-cong-ty.chn"
    if report_type.upper() == 'CF':
        return f"https://s.cafef.vn/bao-cao-tai-chinh/{ticker}/CashFlow/{year}/0/0/0/ket-qua-hoat-dong-kinh-doanh-cong-ty.chn"

    return ""

def get_excel_data(driver, ticker, from_year, to_year, report_type):
    year = to_year

    data = {}
    criteria_names = {}
    runable = True
    while(runable):
        url = get_report_url(ticker, year, report_type)
        print(url)
        driver.get(url)
        driver.implicitly_wait(5)


        if 'criteria' not in criteria_names:
            name_elements = year_col_process(driver,1)
            criteria_names['criteria'] = get_elements_value(name_elements, False)

        year_elements = driver.find_elements(By.XPATH, "//*[@id='tblGridData']/tbody/tr/td")
        index_cols = {}
        i = 1
        for item in year_elements:
            str_item = item.text.strip()
            if str_item.isnumeric():
                index_cols[str_item]=i
            i+=1

        index_cols = dict(sorted(index_cols.items(), reverse=True))

        y = 0
        for key in index_cols:
            col = index_cols[key]
            items = year_col_process(driver,col)
            data[key] = get_elements_value(items)
            print(key)
            y = int(key)
            if y == from_year:
                runable = False
                break
        
        if runable == True:
            year = y - 1


    data = dict(sorted(data.items())) #short dict 
    data = criteria_names | data  #merge two dict into one
    df = pd.DataFrame(data)

    save_as = f"{ticker}_{report_type}.xlsx"
    writer = pd.ExcelWriter(save_as,
                            engine='xlsxwriter',
                            engine_kwargs={'options': {'string_to_numbers': True}})
    df.to_excel(writer, sheet_name = report_type, index = False)
    writer.close()

    return 1

### Crawl Data

In [29]:
# service = ChromeService(executable_path=r"C:\Users\vietl\.wdm\drivers\chromedriver\win64\119.0.6045.105\chromedriver-win32/chromedriver.exe")
# driver_manual = webdriver.Chrome(service=service)

# ticker = 'FPT'
# report_type = 'BS'

# year = 2022
# look_back = 5
# from_year = 2022 - look_back + 1

# data = {}
# criteria_names = {}
# runable = True
# while(runable):
#     url = get_report_url(ticker, year, report_type)
#     print(url)
#     driver_manual.get(url)
#     driver_manual.implicitly_wait(5)


#     if 'criteria' not in criteria_names:
#         name_elements = year_col_process(driver_manual,1)
#         criteria_names['criteria'] = get_elements_value(name_elements, False)

#     year_elements = driver_manual.find_elements(By.XPATH, "//*[@id='tblGridData']/tbody/tr/td")
#     index_cols = {}
#     i = 1
#     for item in year_elements:
#         str_item = item.text.strip()
#         if str_item.isnumeric():
#             index_cols[str_item]=i
#         i+=1

#     index_cols = dict(sorted(index_cols.items(), reverse=True))

#     y = 0
#     for key in index_cols:
#         col = index_cols[key]
#         items = year_col_process(driver_manual,col)
#         data[key] = get_elements_value(items)
#         print(key)
#         y = int(key)
#         if y == from_year:
#             runable = False
#             break
    
#     if runable == True:
#         year = y - 1


# data = dict(sorted(data.items())) #short dict 
# data = criteria_names | data  #merge two dict into one
# df = pd.DataFrame(data)

# save_as = f"{ticker}_{report_type}.xlsx"
# writer = pd.ExcelWriter(save_as,
#                         engine='xlsxwriter',
#                         engine_kwargs={'options': {'string_to_numbers': True}})
# df.to_excel(writer, sheet_name = report_type, index = False)
# writer.close()

In [30]:
import os

In [ ]:
service = ChromeService(executable_path=r"C:\Users\vietl\.wdm\drivers\chromedriver\win64\119.0.6045.105\chromedriver-win32/chromedriver.exe")
driver_manual = webdriver.Chrome(service=service)


In [32]:
TICKER = 'FPT'
TO_YEAR = 2022
LOOK_BACK = 5
FROM_YEAR = TO_YEAR - LOOK_BACK + 1

report_types = ['BS','IS','CF']
for report in report_types:
    get_excel_data(driver_manual, TICKER, FROM_YEAR, TO_YEAR, report)

driver_manual.quit()
os.startfile(os.getcwd())

https://s.cafef.vn/bao-cao-tai-chinh/FPT/BSheet/2022/0/0/0/bao-cao-tai-chinh-cong-ty.chn
2022
2021
2020
2019
https://s.cafef.vn/bao-cao-tai-chinh/FPT/BSheet/2018/0/0/0/bao-cao-tai-chinh-cong-ty.chn
2018
https://s.cafef.vn/bao-cao-tai-chinh/FPT/IncSta/2022/0/0/0/ket-qua-hoat-dong-kinh-doanh-cong-ty.chn
2022
2021
2020
2019
https://s.cafef.vn/bao-cao-tai-chinh/FPT/IncSta/2018/0/0/0/ket-qua-hoat-dong-kinh-doanh-cong-ty.chn
2018
https://s.cafef.vn/bao-cao-tai-chinh/FPT/CashFlow/2022/0/0/0/ket-qua-hoat-dong-kinh-doanh-cong-ty.chn
2022
2021
2020
2019
https://s.cafef.vn/bao-cao-tai-chinh/FPT/CashFlow/2018/0/0/0/ket-qua-hoat-dong-kinh-doanh-cong-ty.chn
2018
